# библиотеки + настройка спарка

In [1]:
# from dsps_modules import krb
# krb.kinit('df')

In [2]:
import os
import sys

os.environ['SPARK_MAJOR_VERSION'] = '2'
os.environ['SPARK_HOME'] = '/usr/sdp/current/spark2-client/'
python_path = '/opt/cloudera/parcels/PYENV.ZNO20008661/bin/python'
os.environ['PYSPARK_DRIVER'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'

sys.path.insert(0, '/usr/sdp/current/spark2-client/python/')

sys.path.insert(0, '/usr/sdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip')
from pyspark import SparkContext, SparkConf, HiveContext

conf = SparkConf().setAppName('XSR')\
    .setMaster("yarn")\
    .set('spark.executor.cores', '5')\
    .set('spark.executor.memory', '21g')\
    .set('spark.executor.memoryOverhead', '4g')\
    .set('spark.driver.memory', '30g')\
    .set('spark.driver.maxResultSize', '30g')\
    .set('spark.hive.mapred.supports.subdirectories', 'true')\
    .set('spark.hadoop.mapreduce.input.fileinputformat.input.dir.recursive', 't')\
    .set('spark.shuffle.service.enabled', 'true')\
    .set('spark.dynamicAllocation.enabled', 'true')\
    .set('spark.dynamicAllocation.executorIdleTimeout', '120s')\
    .set('spark.dynamicAllocation.cachedExecutorIdleTimeout', '600s')\
    .set('spark.dynamicAllocation.initialExecutors', '3')\
    .set('spark.dynamicAllocation.maxExecutors', '32')\
    .set('spark.port.maxRetries', '150')\
    .set('spark.executor.instances', '3')\
    .set('spark.debug.maxToStringFields', '1000')\
    .set('spark.sql.broadcastTimeout', '3600')


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .appName("denzel-top")\
    .master("yarn-client")\
    .config(conf=conf)\
    .getOrCreate()

# витрина

In [4]:
df = spark.read.table("prx_xsr_crm_250722.client_db")

# поиск по epk_id

In [1]:
df_epk = df[ (df['act_flg'] == 'Y') & (df['client_id'].isin([  ***** ])) ] \
    .select('col_1', 
            'col_2', 
            'col_3') 
df_epk_pandas = df_epk.toPandas()

df_epk_pandas.head(1000)

# join разных таблиц

In [ ]:
df_mcc_1 = spark.read.table("xsr_txn_card.mcc")
df_mcc_2 = spark.read.table("xsr_txn_card.mcc_subgroup")
df_mcc_3 = spark.read.table("xsr_txn_card.mcc_group")

df_mcc_temp = df_mcc_1.join(df_mcc_2, df_mcc_1.mcc_subgroup_id == df_mcc_2.mcc_subgroup_id, 'inner')\
                .select('mcc_code', 'mcc_group_id')
df_mcc = df_mcc_temp.join(df_mcc_3, df_mcc_temp.mcc_group_id == df_mcc_3.mcc_group_id, 'inner')\
                .select('mcc_code',  'mcc_group_name').cache()

a = df_mcc.toPandas()
a.head(3)

# остановка spark-сессии

In [6]:
spark.stop()